In [1]:
%cd /home/slav/ai/claim-rank
!export PYTHONPATH=.
%load_ext autoreload
%autoreload 2

/home/slav/ai/claim-rank


In [2]:
from tempfile import NamedTemporaryFile
import numpy as np
from src.models.keras.model_runner import *


/home/slav/anaconda3/envs/ai/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
from keras.layers import Input, Embedding, Conv1D, Dense
from keras.models import Model, load_model
from keras.layers import Input, Dense, Dropout
from keras import optimizers
from sklearn.metrics import average_precision_score

In [42]:
from keras.layers import *
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping
from keras.optimizers import *
from keras.models import Model
from keras.initializers import *
from keras import regularizers

In [4]:
def load_crossvalidation_data(name='crossval'):
    folds = np.load('folds/' + name + '.npy')
    return folds;

In [72]:
def create_baseline_model(in_count, out_count):
    input_layer = Input(shape=(in_count,))
    x = Dense(200, kernel_initializer='normal', activation='relu',
                                       kernel_regularizer=regularizers.l2(0.002))(input_layer)
    
    outputs = list(map(lambda i: Dense(2, kernel_initializer='normal', activation='softmax', 
                                       kernel_regularizer=regularizers.l2(0.002),
                                       name='pred_' + str(i))(x), range(out_count)))
    
    opt=SGD(lr=0.001, momentum=True, nesterov=0.9, decay=0)
    
    model = Model(inputs=[input_layer], outputs=outputs)
    model.compile(optimizer=opt, 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model;

In [63]:
def run_model(X_train, X_test, train_target, test_targets, create_model = create_baseline_model, epochs=100):
    model = create_model(X_train.shape[1], 1)
    model.fit(X_train, [[0,1] if t==0 else [1,0] for t in train_target[0]], epochs=epochs,
              validation_split=0.05, batch_size=50, verbose=1)
    return [average_precision_score(test_targets[j], model.predict(X_test)[:, 0])
         for j in range(len(test_targets))]
    

In [8]:
def run_model_multiple(X_train, X_test, train_targets, test_targets, log_indexes, create_model = create_baseline_model, epochs=20):
    model = create_model(X_train.shape[1], len(train_targets))
    
    
    model.fit(X_train, train_targets, epochs=epochs, batch_size=550, verbose=0)
    return [average_precision_score(test_targets[j], model.predict(X_test)[i][:, 0])
            for j, i in enumerate(log_indexes)]

In [9]:
def cross_validate(folds, train_targets, test_targets):
    models_sums = [0 for _ in test_targets]
    for fold in folds:
        X_train, X_test, y_train, y_test = fold
        y_train = list(y_train[:, train_targets].T)
        y_test = y_test[:, test_targets].T
        models = run_model(X_train, X_test, y_train, y_test)
        models_sums = [prev + curr for prev, curr in zip(models_sums, models)]

    print(np.array(models_sums) / len(folds))

In [10]:
def cross_validate_multiple(folds, train_targets, test_targets, log_indexes):
    models_sums = [0 for _ in test_targets]
    for fold in folds:
        X_train, X_test, y_train, y_test = fold
        y_train = list(y_train[:, train_targets].T)
        y_test = y_test[:, test_targets].T
        models = run_model_multiple(X_train, X_test, y_train, y_test, log_indexes)
        models_sums = [prev + curr for prev, curr in zip(models_sums, models)]

    print(np.array(models_sums) / len(folds))

In [11]:
folds = load_crossvalidation_data(name='serialized')


In [73]:
init_session(use_gpu=False)
cross_validate([folds[0]], [6], [6])

Train on 3860 samples, validate on 204 samples
Epoch 1/100
3860/3860 [==============================] - 3s 837us/step - loss: 0.9119 - acc: 0.9477 - val_loss: 0.6067 - val_acc: 0.9902
Epoch 2/100
3860/3860 [==============================] - 0s 77us/step - loss: 0.7839 - acc: 0.9492 - val_loss: 0.5762 - val_acc: 0.9902
Epoch 3/100
3860/3860 [==============================] - 0s 77us/step - loss: 0.7237 - acc: 0.9492 - val_loss: 0.6250 - val_acc: 0.9902
Epoch 4/100
3860/3860 [==============================] - 0s 76us/step - loss: 0.6603 - acc: 0.9492 - val_loss: 0.4850 - val_acc: 0.9902
Epoch 5/100
3860/3860 [==============================] - 0s 79us/step - loss: 0.6162 - acc: 0.9492 - val_loss: 0.4762 - val_acc: 0.9902
Epoch 6/100
3860/3860 [==============================] - 0s 77us/step - loss: 0.5784 - acc: 0.9492 - val_loss: 0.3970 - val_acc: 0.9902
Epoch 7/100
3860/3860 [==============================] - 0s 75us/step - loss: 0.5411 - acc: 0.9492 - val_loss: 0.4013 - val_acc: 0.9902


In [ ]:
for i in range(1,10):
    cross_validate(folds, [i], [i])

In [ ]:
cross_validate_multiple(folds, range(10), range(1, 10), range(1, 10))

In [17]:
init_session(use_gpu=False)
cross_validate_multiple(folds, [0, 6], [0, 6], [0, 1])

[0.16473989 0.04739189]


In [ ]:
init_session(use_gpu=False)
cross_validate_multiple(folds, range(10), [0, 6], [0, 6])

In [ ]:
folds = load_crossvalidation_data(name='serialized')
init_session(use_gpu=False)
cross_validate(folds, [6], [6])
cross_validate_multiple(folds, range(10), [0, 6], [0, 6])

In [ ]:
folds = load_crossvalidation_data(name='serialized')
# init_session(use_gpu=False)
cross_validate(folds, [6], [6])
cross_validate_multiple(folds, [0, 6], [0, 6], [0, 1])
cross_validate_multiple(folds, range(10), [0, 6], [0, 6])

In [ ]:
folds = load_crossvalidation_data(name='serialized')
init_session(use_gpu=False)
# cross_validate(folds, [6], [6])
cross_validate_multiple(folds, range(10), [0, 6], [0, 6])